In [1]:
import pandas as pd

In [103]:
poi_df = pd.read_csv('./data/weekly_and_core_with_area.csv')

In [104]:
poi_df = poi_df[[
    'placekey', 'poi_cbg', 'date_range_start', 'date_range_end',
    'raw_visit_counts', 'area_square_feet'
]]

In [105]:
poi_df[
    'contact_density'] = poi_df['raw_visit_counts'] / poi_df['area_square_feet']
poi_df = poi_df.drop(
    columns=['raw_visit_counts', 'area_square_feet', 'date_range_end'])

In [106]:
def repair_dates(df):
    df['date_range_start'] = pd.to_datetime(df['date_range_start'], utc=True)
    df['date_range_start'] = df['date_range_start'].dt.tz_convert('US/Eastern')

repair_dates(poi_df)

In [107]:
poi_df = poi_df.sort_values(by=['poi_cbg', 'date_range_start'])

In [108]:
poi_df.head()

,placekey,poi_cbg,date_range_start,contact_density
2818653,22b-222@627-rwr-4n5,360050002001,2018-12-31 00:00:00-05:00,0.039394
2818654,22b-222@627-rwr-4n5,360050002001,2019-01-07 00:00:00-05:00,0.026984
2818655,22b-222@627-rwr-4n5,360050002001,2019-01-14 00:00:00-05:00,0.019014
2818593,22b-222@627-rwr-4n5,360050002001,2019-01-21 00:00:00-05:00,0.022316
2818656,22b-222@627-rwr-4n5,360050002001,2019-01-28 00:00:00-05:00,0.018445


In [109]:
poi_df.to_csv(f'./exports/poi_weekly.csv', index=False)

In [110]:
dc_df = pd.read_csv('./data/device_count_nyc_weekly.csv')

In [122]:
dc_df = dc_df.rename(columns={'origin_census_block_group': 'home_cbg'})
dc_df = dc_df.drop(columns=['date_range_end'])

In [112]:
repair_dates(dc_df)

In [113]:
trips_df = pd.read_csv('./data/weekly_trips_by_cbg.csv')

In [119]:
trips_df = trips_df.rename(columns={'visitor_home_cbg_id': 'home_cbg'})
trips_df = trips_df.drop(columns=['poi_cbg', 'date_range_end', 'distance'])

repair_dates(trips_df)
trips_df

,placekey,date_range_start,home_cbg,visitor_count
0,222-222@627-s94-nwk,2020-12-21 00:00:00-05:00,360470399002,4
1,222-222@627-s94-nwk,2020-12-21 00:00:00-05:00,360471142022,4
2,222-222@627-s94-nwk,2020-12-21 00:00:00-05:00,360470395002,4
3,222-222@627-s94-nwk,2020-12-21 00:00:00-05:00,360470399003,4
4,222-222@627-s94-nwk,2021-01-11 00:00:00-05:00,360470397003,8
...,...,...,...,...
27839722,zzy-224@627-s8r-bx5,2020-03-09 00:00:00-04:00,360810548001,10
27839723,zzy-224@627-s8r-bx5,2020-03-09 00:00:00-04:00,360810291004,4
27839724,zzy-224@627-s8r-bx5,2020-03-09 00:00:00-04:00,360810028001,4
27839725,zzy-224@627-s8r-bx5,2020-03-09 00:00:00-04:00,360610152001,4


In [134]:
merged_df = trips_df.merge(dc_df,
                           on=['home_cbg', 'date_range_start'],
                           how='inner')
merged_df

,placekey,date_range_start,home_cbg,visitor_count,device_count
0,222-222@627-s94-nwk,2020-12-21 00:00:00-05:00,360470399002,4,192
1,22m-222@627-s96-pqf,2020-12-21 00:00:00-05:00,360470399002,5,192
2,22d-222@627-s7r-49z,2020-12-21 00:00:00-05:00,360470399002,4,192
3,23s-222@627-wc7-zs5,2020-12-21 00:00:00-05:00,360470399002,4,192
4,224-222@627-s94-whq,2020-12-21 00:00:00-05:00,360470399002,4,192
...,...,...,...,...,...
27825633,223-222@627-wbv-jd9,2020-12-21 00:00:00-05:00,360610133001,4,74
27825634,22f-222@627-wc8-435,2020-09-14 00:00:00-04:00,360610075003,4,95
27825635,232-222@627-wc8-4n5,2020-06-22 00:00:00-04:00,360610063005,4,52
27825636,23f-222@627-s4n-tsq,2020-06-15 00:00:00-04:00,360610148011,4,217


In [135]:
population_df = pd.read_csv('./cbg_attr_and_cluster.csv')

In [136]:
population_df = population_df[['census_block_group', 'Total Population']]
population_df = population_df.rename(columns={
    'census_block_group': 'home_cbg',
    'Total Population': 'population'
})

In [137]:
population_df

,home_cbg,population
0,360050001000,0
1,360050001001,6864
2,360050002000,0
3,360050002001,1744
4,360050002002,1569
...,...,...
6488,360850319022,1667
6489,360850319023,1945
6490,360850323000,0
6491,360850323001,1073


In [138]:
est_df = merged_df.merge(population_df, on=['home_cbg'], how='inner')

In [139]:
est_df['estimated_visitor_count'] = (est_df['visitor_count'] *
                                     est_df['population'] /
                                     est_df['device_count'])
est_df

,placekey,date_range_start,home_cbg,visitor_count,device_count,population,estimated_visitor_count
0,222-222@627-s94-nwk,2020-12-21 00:00:00-05:00,360470399002,4,192,1487,30.979167
1,22m-222@627-s96-pqf,2020-12-21 00:00:00-05:00,360470399002,5,192,1487,38.723958
2,22d-222@627-s7r-49z,2020-12-21 00:00:00-05:00,360470399002,4,192,1487,30.979167
3,23s-222@627-wc7-zs5,2020-12-21 00:00:00-05:00,360470399002,4,192,1487,30.979167
4,224-222@627-s94-whq,2020-12-21 00:00:00-05:00,360470399002,4,192,1487,30.979167
...,...,...,...,...,...,...,...
27825633,22b-222@627-tmm-rp9,2020-12-28 00:00:00-05:00,360810934010,4,19,0,0.000000
27825634,zzy-222@627-wbz-yy9,2019-03-11 00:00:00-04:00,360470044000,4,18,0,0.000000
27825635,22h-222@627-wh2-tvz,2019-04-01 00:00:00-04:00,360470044000,4,14,0,0.000000
27825636,222-222@627-rwn-b6k,2020-12-21 00:00:00-05:00,360050019005,6,9,0,0.000000


In [140]:
est_df = est_df.drop(columns=['device_count', 'population'])

In [141]:
# Sum visitor count and estimated visitor count per CBG.
total_df = est_df.groupby(by=['home_cbg', 'date_range_start']).agg({'visitor_count': 'sum', 'estimated_visitor_count': 'sum'}).reset_index()
total_df

,home_cbg,date_range_start,visitor_count,estimated_visitor_count
0,360050001000,2018-12-31 00:00:00-05:00,8,0.0
1,360050001000,2019-01-07 00:00:00-05:00,12,0.0
2,360050001000,2019-01-14 00:00:00-05:00,4,0.0
3,360050001000,2019-01-21 00:00:00-05:00,8,0.0
4,360050001000,2019-02-04 00:00:00-05:00,4,0.0
...,...,...,...,...
688099,360859901000,2020-11-16 00:00:00-05:00,4,0.0
688100,360859901000,2021-01-18 00:00:00-05:00,8,0.0
688101,360859901000,2021-02-01 00:00:00-05:00,4,0.0
688102,360859901000,2021-02-08 00:00:00-05:00,4,0.0


In [143]:
pct_df = est_df.merge(total_df,
                      how='left',
                      on=['home_cbg', 'date_range_start'],
                      suffixes=('_to_poi', '_to_all'))
pct_df

,placekey,date_range_start,home_cbg,visitor_count_to_poi,estimated_visitor_count_to_poi,visitor_count_to_all,estimated_visitor_count_to_all
0,222-222@627-s94-nwk,2020-12-21 00:00:00-05:00,360470399002,4,30.979167,41,317.536458
1,22m-222@627-s96-pqf,2020-12-21 00:00:00-05:00,360470399002,5,38.723958,41,317.536458
2,22d-222@627-s7r-49z,2020-12-21 00:00:00-05:00,360470399002,4,30.979167,41,317.536458
3,23s-222@627-wc7-zs5,2020-12-21 00:00:00-05:00,360470399002,4,30.979167,41,317.536458
4,224-222@627-s94-whq,2020-12-21 00:00:00-05:00,360470399002,4,30.979167,41,317.536458
...,...,...,...,...,...,...,...
27825633,22b-222@627-tmm-rp9,2020-12-28 00:00:00-05:00,360810934010,4,0.000000,4,0.000000
27825634,zzy-222@627-wbz-yy9,2019-03-11 00:00:00-04:00,360470044000,4,0.000000,4,0.000000
27825635,22h-222@627-wh2-tvz,2019-04-01 00:00:00-04:00,360470044000,4,0.000000,4,0.000000
27825636,222-222@627-rwn-b6k,2020-12-21 00:00:00-05:00,360050019005,6,0.000000,6,0.000000


In [145]:
pct_df['pct_visitor_count'] = (pct_df['visitor_count_to_poi'] /
                               pct_df['visitor_count_to_all'])
pct_df['pct_estimated_visitor_count'] = (
    pct_df['estimated_visitor_count_to_poi'] /
    pct_df['estimated_visitor_count_to_all'])

In [146]:
pct_df = pct_df.sort_values(by=['home_cbg', 'date_range_start'])

In [148]:
# Sum values and percentages by home_cbg to verify they equal totals/100%.
pct_check_df = pct_df.groupby(by=['home_cbg', 'date_range_start']).agg({
    'visitor_count_to_poi': 'sum',
    'estimated_visitor_count_to_poi': 'sum',
    'visitor_count_to_all': 'first',
    'estimated_visitor_count_to_all': 'first',
    'pct_visitor_count': 'sum',
    'pct_estimated_visitor_count': 'sum',
}).reset_index()
pct_check_df

,home_cbg,date_range_start,visitor_count_to_poi,estimated_visitor_count_to_poi,visitor_count_to_all,estimated_visitor_count_to_all,pct_visitor_count,pct_estimated_visitor_count
0,360050001000,2018-12-31 00:00:00-05:00,8,0.0,8,0.0,1.0,0.0
1,360050001000,2019-01-07 00:00:00-05:00,12,0.0,12,0.0,1.0,0.0
2,360050001000,2019-01-14 00:00:00-05:00,4,0.0,4,0.0,1.0,0.0
3,360050001000,2019-01-21 00:00:00-05:00,8,0.0,8,0.0,1.0,0.0
4,360050001000,2019-02-04 00:00:00-05:00,4,0.0,4,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...
688099,360859901000,2020-11-16 00:00:00-05:00,4,0.0,4,0.0,1.0,0.0
688100,360859901000,2021-01-18 00:00:00-05:00,8,0.0,8,0.0,1.0,0.0
688101,360859901000,2021-02-01 00:00:00-05:00,4,0.0,4,0.0,1.0,0.0
688102,360859901000,2021-02-08 00:00:00-05:00,4,0.0,4,0.0,1.0,0.0


In [149]:
pct_df = pct_df.fillna(0)
pct_df = pct_df.rename(columns={'visitor_count_to_poi': 'visitor_count', 'estimated_visitor_count_to_poi': 'estimated_visitor_count'})
pct_df = pct_df.drop(columns=['visitor_count_to_all', 'estimated_visitor_count_to_all'])

In [150]:
pct_df

,placekey,date_range_start,home_cbg,visitor_count,estimated_visitor_count,pct_visitor_count,pct_estimated_visitor_count
27825396,22t-222@627-s7m-rtv,2018-12-31 00:00:00-05:00,360050001000,4,0.0,0.500000,0.0
27825397,zzy-222@627-sbb-3dv,2018-12-31 00:00:00-05:00,360050001000,4,0.0,0.500000,0.0
27825393,22t-222@627-s7m-rtv,2019-01-07 00:00:00-05:00,360050001000,4,0.0,0.333333,0.0
27825394,222-222@627-rzq-cyv,2019-01-07 00:00:00-05:00,360050001000,4,0.0,0.333333,0.0
27825395,226-222@627-s6g-vfz,2019-01-07 00:00:00-05:00,360050001000,4,0.0,0.333333,0.0
...,...,...,...,...,...,...,...
27824886,zzw-222@627-vvc-jy9,2021-01-18 00:00:00-05:00,360859901000,4,0.0,0.500000,0.0
27824887,22b-222@627-wfb-c5z,2021-01-18 00:00:00-05:00,360859901000,4,0.0,0.500000,0.0
27824902,zzw-224@627-vvc-tqf,2021-02-01 00:00:00-05:00,360859901000,4,0.0,1.000000,0.0
27824900,zzw-224@627-vvc-tqf,2021-02-08 00:00:00-05:00,360859901000,4,0.0,1.000000,0.0


In [153]:
cd_df = pct_df.merge(poi_df, how='inner', on=['placekey', 'date_range_start'])


,placekey,date_range_start,home_cbg,visitor_count,estimated_visitor_count,pct_visitor_count,pct_estimated_visitor_count,poi_cbg,contact_density
0,22t-222@627-s7m-rtv,2018-12-31 00:00:00-05:00,360050001000,4,0.000000,0.500000,0.000000,360810635002,0.269690
1,22t-222@627-s7m-rtv,2018-12-31 00:00:00-05:00,360470437004,4,9.974576,0.018433,0.018433,360810635002,0.269690
2,22t-222@627-s7m-rtv,2018-12-31 00:00:00-05:00,360470493005,4,8.588745,0.020101,0.020101,360810635002,0.269690
3,22t-222@627-s7m-rtv,2018-12-31 00:00:00-05:00,360471134003,4,2.479008,0.008333,0.008333,360810635002,0.269690
4,22t-222@627-s7m-rtv,2018-12-31 00:00:00-05:00,360471196002,4,0.212593,0.001224,0.001224,360810635002,0.269690
...,...,...,...,...,...,...,...,...,...
27825633,zzw-224@627-wbz-r6k,2021-02-01 00:00:00-05:00,360850323001,4,6.202312,0.031008,0.031008,360610015021,NaN
27825634,224-224@627-wbz-qzz,2021-02-01 00:00:00-05:00,360850323001,4,6.202312,0.031008,0.031008,360610015021,0.002361
27825635,zzw-227@627-wbz-qzz,2021-02-01 00:00:00-05:00,360850323001,4,6.202312,0.031008,0.031008,360610015021,NaN
27825636,zzw-225@627-wbz-rc5,2021-02-08 00:00:00-05:00,360850323001,4,6.036568,0.023529,0.023529,360610015021,0.000145


In [154]:
cd_df = cd_df.fillna(0)
cd_df

,placekey,date_range_start,home_cbg,visitor_count,estimated_visitor_count,pct_visitor_count,pct_estimated_visitor_count,poi_cbg,contact_density
0,22t-222@627-s7m-rtv,2018-12-31 00:00:00-05:00,360050001000,4,0.000000,0.500000,0.000000,360810635002,0.269690
1,22t-222@627-s7m-rtv,2018-12-31 00:00:00-05:00,360470437004,4,9.974576,0.018433,0.018433,360810635002,0.269690
2,22t-222@627-s7m-rtv,2018-12-31 00:00:00-05:00,360470493005,4,8.588745,0.020101,0.020101,360810635002,0.269690
3,22t-222@627-s7m-rtv,2018-12-31 00:00:00-05:00,360471134003,4,2.479008,0.008333,0.008333,360810635002,0.269690
4,22t-222@627-s7m-rtv,2018-12-31 00:00:00-05:00,360471196002,4,0.212593,0.001224,0.001224,360810635002,0.269690
...,...,...,...,...,...,...,...,...,...
27825633,zzw-224@627-wbz-r6k,2021-02-01 00:00:00-05:00,360850323001,4,6.202312,0.031008,0.031008,360610015021,0.000000
27825634,224-224@627-wbz-qzz,2021-02-01 00:00:00-05:00,360850323001,4,6.202312,0.031008,0.031008,360610015021,0.002361
27825635,zzw-227@627-wbz-qzz,2021-02-01 00:00:00-05:00,360850323001,4,6.202312,0.031008,0.031008,360610015021,0.000000
27825636,zzw-225@627-wbz-rc5,2021-02-08 00:00:00-05:00,360850323001,4,6.036568,0.023529,0.023529,360610015021,0.000145


In [156]:
cd_df['cdi'] = cd_df['contact_density'] * cd_df['estimated_visitor_count']
cd_df = cd_df.drop(columns=['poi_cbg', 'contact_density'])
cd_df

,placekey,date_range_start,home_cbg,visitor_count,estimated_visitor_count,pct_visitor_count,pct_estimated_visitor_count,cdi
0,22t-222@627-s7m-rtv,2018-12-31 00:00:00-05:00,360050001000,4,0.000000,0.500000,0.000000,0.000000
1,22t-222@627-s7m-rtv,2018-12-31 00:00:00-05:00,360470437004,4,9.974576,0.018433,0.018433,2.690041
2,22t-222@627-s7m-rtv,2018-12-31 00:00:00-05:00,360470493005,4,8.588745,0.020101,0.020101,2.316296
3,22t-222@627-s7m-rtv,2018-12-31 00:00:00-05:00,360471134003,4,2.479008,0.008333,0.008333,0.668563
4,22t-222@627-s7m-rtv,2018-12-31 00:00:00-05:00,360471196002,4,0.212593,0.001224,0.001224,0.057334
...,...,...,...,...,...,...,...,...
27825633,zzw-224@627-wbz-r6k,2021-02-01 00:00:00-05:00,360850323001,4,6.202312,0.031008,0.031008,0.000000
27825634,224-224@627-wbz-qzz,2021-02-01 00:00:00-05:00,360850323001,4,6.202312,0.031008,0.031008,0.014642
27825635,zzw-227@627-wbz-qzz,2021-02-01 00:00:00-05:00,360850323001,4,6.202312,0.031008,0.031008,0.000000
27825636,zzw-225@627-wbz-rc5,2021-02-08 00:00:00-05:00,360850323001,4,6.036568,0.023529,0.023529,0.000872


In [ ]:
cd_df.to_csv(f'./exports/home_weekly.csv', index=False)